

Задание 1. Написать теггер на данных с русским языком

    -   проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации
    -   написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
    -   сравнить все реализованные методы сделать выводы



In [1]:
#nltk.download() 
import nltk
from nltk.tokenize import word_tokenize
import matplotlib
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
#from nltk.corpus import brown
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger

In [5]:
!pip install pyconll

In [6]:
import pyconll

In [13]:
!wget -O ./data/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu

--2023-01-29 21:43:04--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 40736581 (39M) [text/plain]
Сохранение в: ‘./data/ru_syntagrus-ud-train.conllu’

./data/ru_syntagrus 100%[===================>]  38,85M  9,02MB/s    за 4,7s    

2023-01-29 21:43:09 (8,21 MB/s) - ‘./data/ru_syntagrus-ud-train.conllu’ сохранён [40736581/40736581]



In [14]:
!wget -O ./data/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2023-01-29 21:44:00--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 14704579 (14M) [text/plain]
Сохранение в: ‘./data/ru_syntagrus-ud-dev.conllu’

./data/ru_syntagrus 100%[===================>]  14,02M  7,05MB/s    за 2,0s    

2023-01-29 21:44:03 (7,05 MB/s) - ‘./data/ru_syntagrus-ud-dev.conllu’ сохранён [14704579/14704579]



In [11]:
!ls

data	     NER-2.ipynb	   NER_rus.ipynb
image	     ner_nn.ipynb	   Pos_teggers.ipynb
NER-1.ipynb  NER_restaurant.ipynb  ru_syntagrus-ud-train-a.conllu


In [15]:
full_train = pyconll.load_from_file('data/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('data/ru_syntagrus-ud-dev.conllu')

In [16]:
for sent in full_test[23:24]:
    for token in sent:
        print(token.form, token.upos)
    print()

Однако ADV
при ADP
включении NOUN
метода NOUN
генерации NOUN
случайных ADJ
чисел NOUN
в ADP
список NOUN
" PUNCT
исходных ADJ
данных NOUN
" PUNCT
, PUNCT
вероятностный ADJ
алгоритм NOUN
становится VERB
подвидом NOUN
обычного ADJ
. PUNCT



In [17]:
for sent in full_test[23:24]:
    for token in sent:
        print(type(token.form))
    print()

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>



In [18]:
# удалил из датасета неинформативную пунктуацию
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent if token.upos not in ['PUNCT']])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent if token.upos not in ['PUNCT']])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent if token.upos not in ['PUNCT']])

In [19]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 194
Наибольшая длина токена 31


In [20]:
MAX_ORIG_TOKEN = sorted((len(token.form), token.form) for sent in full_train for token in sent)

In [22]:
MAX_ORIG_TOKEN[:5]

[(1, '!'), (1, '!'), (1, '!'), (1, '!'), (1, '!')]

In [23]:
sorted(MAX_ORIG_TOKEN, reverse=True)[:20]

[(31, 'профессорско-преподавательского'),
 (30, 'потребительско-патерналистский'),
 (28, 'фаталистически-расслабленное'),
 (27, 'информационно-навигационную'),
 (26, 'художественно-эстетических'),
 (26, 'спектрально-энергетический'),
 (26, 'социально-реабилитационных'),
 (26, 'неинституционализированная'),
 (26, 'национал-социалистического'),
 (26, 'культурно-просветительских'),
 (26, 'Петропавловска-Камчатского'),
 (25, 'социально-психологических'),
 (25, 'партийно-государственнной'),
 (25, 'общественно-экономическую'),
 (25, 'общественно-политического'),
 (25, 'национал-социалистической'),
 (25, 'взаимно-перпендикулярными'),
 (25, 'Национал-социалистическая'),
 (24, 'финансово-олигархической'),
 (24, 'ультранационалистических')]

In [24]:
all_train_texts = [' '.join(token.form for token in sent if token.upos not in ['PUNCT']) for sent in full_train]
print('\n'.join(all_train_texts[:10]))

Анкета
Начальник областного управления связи Семен Еремеевич был человек простой приходил на работу всегда вовремя здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом Муха
В приемной его с утра ожидали посетители кое-кто с важными делами а кое-кто и с такими которые легко можно было решить в нижестоящих инстанциях не затрудняя Семена Еремеевича
Однако стиль работы Семена Еремеевича заключался в том чтобы принимать всех желающих и лично вникать в дело
Приемная была обставлена просто но по-деловому
У двери стоял стол секретарши на столе пишущая машинка с широкой кареткой
В углу висел репродуктор и играло радио для развлечения ожидающих и еще для того чтобы заглушать голос начальника доносившийся из кабинета так как бесспорно среди посетителей могли находиться и случайные люди
Кабинет отличался скромностью присущей Семену Еремеевичу
В глубине стоял широкий письменный стол с бронзовыми чернильницами и перед ним два кожаных кресла
Справа был стол для за

In [25]:
for sent in full_test[6587:6588]:
    for token in sent:
        print(token.form, token.upos)
    print()

Можно ADV
различать VERB
( PUNCT
1 NUM
) PUNCT
большую ADJ
науку NOUN
, PUNCT
( PUNCT
2 NUM
) PUNCT
фундаментальную ADJ
, PUNCT
( PUNCT
3 NUM
) PUNCT
фундаментальную ADJ
социально ADV
ориентированную VERB
( PUNCT
поисковую ADJ
? PUNCT
) PUNCT
None None
и CCONJ
( PUNCT
4 NUM
) PUNCT
прикладную ADJ
науку NOUN
. PUNCT



In [26]:
test_texts = [' '.join(token.form for token in sent if token.form is not None and token.upos is not None and token.upos not in ['PUNCT']) for sent in full_test[6587:6588]]
test_texts

['Можно различать 1 большую науку 2 фундаментальную 3 фундаментальную социально ориентированную поисковую и 4 прикладную науку']

In [27]:
test_texts = [' '.join(token.form for token in sent if token.upos not in ['PUNCT']) for sent in full_test[6588:6589]]
test_texts

['Первая касается важных проектов государственного значения типа ядерного оружия или космоса']

In [28]:
all_test_texts = [' '.join(token.form for token in sent if token.form is not None and token.upos is not None\
                           and token.upos not in ['PUNCT']) for sent in full_test]

In [29]:
all_train_labels = [' '.join(token.upos for token in sent if token.form is not None and token.upos is not None\
                             and token.upos not in ['PUNCT']) for sent in full_train]
all_test_labels = [' '.join(token.upos for token in sent if token.form is not None and token.upos is not None\
                            and token.upos not in ['PUNCT']) for sent in full_test]
print('\n'.join(all_train_texts[:10]))

Анкета
Начальник областного управления связи Семен Еремеевич был человек простой приходил на работу всегда вовремя здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом Муха
В приемной его с утра ожидали посетители кое-кто с важными делами а кое-кто и с такими которые легко можно было решить в нижестоящих инстанциях не затрудняя Семена Еремеевича
Однако стиль работы Семена Еремеевича заключался в том чтобы принимать всех желающих и лично вникать в дело
Приемная была обставлена просто но по-деловому
У двери стоял стол секретарши на столе пишущая машинка с широкой кареткой
В углу висел репродуктор и играло радио для развлечения ожидающих и еще для того чтобы заглушать голос начальника доносившийся из кабинета так как бесспорно среди посетителей могли находиться и случайные люди
Кабинет отличался скромностью присущей Семену Еремеевичу
В глубине стоял широкий письменный стол с бронзовыми чернильницами и перед ним два кожаных кресла
Справа был стол для за

# проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации

In [31]:
unigram_tagger = UnigramTagger(fdata_train)

display(unigram_tagger.tag(fdata_sent_test[100]), unigram_tagger.evaluate(fdata_test))

[('Это', 'PRON'),
 ('сочинение', None),
 ('известно', 'ADJ'),
 ('во', 'ADP'),
 ('многих', 'NUM'),
 ('вариантах', 'NOUN'),
 ('самые', 'ADJ'),
 ('ранние', 'ADJ'),
 ('из', 'ADP'),
 ('них', 'PRON'),
 ('почти', 'ADV'),
 ('на', 'ADP'),
 ('сто', 'NUM'),
 ('лет', 'NOUN'),
 ('старше', 'ADJ'),
 ('и', 'CCONJ'),
 ('восходит', 'VERB'),
 ('к', 'ADP'),
 ('ещё', 'ADV'),
 ('более', 'ADV'),
 ('древним', 'ADJ'),
 ('рукописям', None),
 ('XVI', 'NUM'),
 ('в', 'ADP')]

0.7824266100768464

In [33]:
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)

display(bigram_tagger.tag(fdata_sent_test[100]), bigram_tagger.evaluate(fdata_test))

[('Это', 'PRON'),
 ('сочинение', None),
 ('известно', 'ADJ'),
 ('во', 'ADP'),
 ('многих', 'NUM'),
 ('вариантах', 'NOUN'),
 ('самые', 'ADJ'),
 ('ранние', 'ADJ'),
 ('из', 'ADP'),
 ('них', 'PRON'),
 ('почти', 'ADV'),
 ('на', 'ADP'),
 ('сто', 'NUM'),
 ('лет', 'NOUN'),
 ('старше', 'ADJ'),
 ('и', 'CCONJ'),
 ('восходит', 'VERB'),
 ('к', 'ADP'),
 ('ещё', 'ADV'),
 ('более', 'ADV'),
 ('древним', 'ADJ'),
 ('рукописям', None),
 ('XVI', 'NUM'),
 ('в', 'ADP')]

0.7879650172020193

In [34]:
trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
display(trigram_tagger.tag(fdata_sent_test[100]), trigram_tagger.evaluate(fdata_test))

[('Это', 'PRON'),
 ('сочинение', None),
 ('известно', 'ADJ'),
 ('во', 'ADP'),
 ('многих', 'NUM'),
 ('вариантах', 'NOUN'),
 ('самые', 'ADJ'),
 ('ранние', 'ADJ'),
 ('из', 'ADP'),
 ('них', 'PRON'),
 ('почти', 'ADV'),
 ('на', 'ADP'),
 ('сто', 'NUM'),
 ('лет', 'NOUN'),
 ('старше', 'ADJ'),
 ('и', 'CCONJ'),
 ('восходит', 'VERB'),
 ('к', 'ADP'),
 ('ещё', 'ADV'),
 ('более', 'ADV'),
 ('древним', 'ADJ'),
 ('рукописям', None),
 ('XVI', 'NUM'),
 ('в', 'ADP')]

0.7869602263592811

In [35]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NN') 
tag = backoff_tagger(fdata_train,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tag.evaluate(fdata_test) 

0.7865100800617344



После удаления пунктуации скор упал на 3%, но оставлю код на будущее.


# 2. написать свой теггер как на занятии, попробовать разные

In [40]:
nltk.download('names')

[nltk_data] Downloading package names to /home/jb/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.


True

In [41]:
from nltk.tag import SequentialBackoffTagger
from nltk.corpus import names

class NamesTagger(SequentialBackoffTagger):
    def __init__(self, *args, **kwargs):
        SequentialBackoffTagger.__init__(self, *args, **kwargs)
        self.name_set = set([n.lower() for n in names.words()])
            
    def choose_tag(self, tokens, index, history):
        word = tokens[index]
        if word.lower() in self.name_set:
             return 'NNP'
        else:
             return None

In [42]:
nt = NamesTagger()
print(nt.tag(['Katya'])) 
print(nt.tag(['Adam'])) 
print(nt.tag(['Window'])) 

[('Katya', 'NNP')]
[('Adam', 'NNP')]
[('Window', None)]


In [43]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    sent_list = [tok for tok in sent if tok[0] is not None and tok[1] is not None and tok[1] not in ['PUNCT']]
    for tok in sent_list:
        train_tok.append(tok[0])
        train_label.append(tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    sent_list = [tok for tok in sent if tok[0] is not None and tok[1] is not None and tok[1] not in ['PUNCT']]
    for tok in sent_list:
        test_tok.append(tok[0])
        test_label.append(tok[1])

In [44]:
# надо бы предлоги почистить еще
nltk.FreqDist(train_tok)

FreqDist({'в': 10635, 'и': 10476, 'не': 5245, 'на': 5196, 'что': 3726, 'с': 3334, 'по': 2043, 'к': 1706, 'как': 1659, 'а': 1622, ...})

In [45]:
display(nltk.FreqDist(train_label))

FreqDist({'NOUN': 103538, 'VERB': 47731, 'ADJ': 43357, 'ADP': 39344, 'ADV': 22733, 'PRON': 18733, 'CCONJ': 15168, 'PROPN': 14855, 'PART': 13556, 'DET': 10781, ...})

In [46]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
#import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [48]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)

test_enc_labels = le.transform(test_label)

In [49]:
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM',
       'PART', 'PRON', 'PROPN', 'SCONJ', 'SYM', 'VERB', 'X'], dtype='<U5')

# HashingVectorizer

In [50]:
hvectorizer = HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=100)

In [51]:
X_train = hvectorizer.fit_transform(train_tok)

In [52]:
X_test = hvectorizer.transform(test_tok)

In [53]:
lr = LogisticRegression(random_state=0, max_iter=100)
lr.fit(X_train, train_enc_labels)

LogisticRegression(random_state=0)

In [54]:
pred = lr.predict(X_test)

In [55]:
# INTJ and X классы по нулям
print(classification_report(test_enc_labels, pred))

              precision    recall  f1-score   support

           0       0.58      0.52      0.55     15103
           1       0.77      0.91      0.83     13717
           2       0.47      0.25      0.33      7783
           3       0.81      0.95      0.88      1390
           4       0.86      0.96      0.91      5672
           5       0.60      0.62      0.61      4265
           6       0.00      0.00      0.00        24
           7       0.56      0.71      0.63     36238
           8       0.58      0.50      0.54      1734
           9       0.76      0.62      0.69      5125
          10       0.59      0.54      0.56      7444
          11       0.54      0.12      0.19      5473
          12       0.76      0.89      0.82      2865
          13       1.00      0.69      0.82        62
          14       0.63      0.54      0.58     17110
          15       0.00      0.00      0.00       134

    accuracy                           0.63    124139
   macro avg       0.59   

In [56]:
hvectorizer = HashingVectorizer(ngram_range=(1, 2), analyzer='word', n_features=100)

In [57]:
X_train = hvectorizer.fit_transform(train_tok)
X_test = hvectorizer.transform(test_tok)

In [58]:
lr = LogisticRegression(random_state=0, max_iter=100)
lr.fit(X_train, train_enc_labels)

LogisticRegression(random_state=0)

In [59]:
pred = lr.predict(X_test)

In [60]:
print(classification_report(test_enc_labels, pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     15103
           1       0.60      0.37      0.46     13717
           2       0.35      0.23      0.28      7783
           3       0.32      0.34      0.33      1390
           4       0.53      0.12      0.19      5672
           5       0.30      0.13      0.18      4265
           6       0.00      0.00      0.00        24
           7       0.33      0.82      0.47     36238
           8       0.00      0.00      0.00      1734
           9       0.58      0.57      0.58      5125
          10       0.42      0.38      0.40      7444
          11       0.16      0.02      0.04      5473
          12       0.55      0.70      0.61      2865
          13       0.00      0.00      0.00        62
          14       0.28      0.02      0.04     17110
          15       0.00      0.00      0.00       134

    accuracy                           0.37    124139
   macro avg       0.28   

# CountVectorizer

In [61]:
count_vectorizer = CountVectorizer(ngram_range=(1, 3), analyzer='char', binary=False)
# Создаем the Bag-of-Words модель
X_train = count_vectorizer.fit_transform(train_tok)

In [62]:
list(count_vectorizer.vocabulary_.items())[:10]

[('а', 2786),
 ('н', 7169),
 ('к', 6165),
 ('е', 4520),
 ('т', 9282),
 ('ан', 3078),
 ('нк', 7335),
 ('ке', 6230),
 ('ет', 4919),
 ('та', 9300)]

In [63]:
X_test = count_vectorizer.transform(test_tok)

In [64]:
lr = LogisticRegression(random_state=0, max_iter=100)
lr.fit(X_train, train_enc_labels)
pred = lr.predict(X_test)

In [65]:
print(classification_report(test_enc_labels, pred))

              precision    recall  f1-score   support

           0       0.91      0.90      0.91     15103
           1       0.98      1.00      0.99     13717
           2       0.90      0.90      0.90      7783
           3       0.81      0.97      0.88      1390
           4       0.88      0.98      0.93      5672
           5       0.89      0.73      0.80      4265
           6       0.32      0.29      0.30        24
           7       0.92      0.95      0.94     36238
           8       0.84      0.89      0.87      1734
           9       0.96      0.76      0.85      5125
          10       0.83      0.90      0.86      7444
          11       0.74      0.60      0.66      5473
          12       0.80      0.91      0.85      2865
          13       1.00      0.73      0.84        62
          14       0.94      0.93      0.94     17110
          15       0.53      0.13      0.20       134

    accuracy                           0.91    124139
   macro avg       0.83   

In [66]:
count_vectorizer = CountVectorizer(ngram_range=(1, 2), analyzer='word', binary=False)
# Создаем the Bag-of-Words модель
X_train = count_vectorizer.fit_transform(train_tok)

In [67]:
X_test = count_vectorizer.transform(test_tok)

In [68]:
lr = LogisticRegression(random_state=0, max_iter=100)
lr.fit(X_train, train_enc_labels)
pred = lr.predict(X_test)

In [69]:
print(classification_report(test_enc_labels, pred))

              precision    recall  f1-score   support

           0       0.94      0.49      0.65     15103
           1       0.98      0.48      0.64     13717
           2       0.94      0.80      0.86      7783
           3       0.82      0.95      0.88      1390
           4       0.89      0.20      0.33      5672
           5       0.84      0.74      0.78      4265
           6       0.00      0.00      0.00        24
           7       0.52      0.99      0.68     36238
           8       0.87      0.61      0.72      1734
           9       0.99      0.73      0.84      5125
          10       0.89      0.78      0.83      7444
          11       0.91      0.18      0.30      5473
          12       0.73      0.98      0.84      2865
          13       0.00      0.00      0.00        62
          14       0.99      0.59      0.74     17110
          15       0.00      0.00      0.00       134

    accuracy                           0.69    124139
   macro avg       0.71   

# TfidfVectorizer

In [70]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3), analyzer='char', binary=False)
# Создаем the Bag-of-Words модель
X_train = tfidf_vectorizer.fit_transform(train_tok)

In [71]:
X_test = tfidf_vectorizer.transform(test_tok)
X_test.shape

(124139, 11981)

In [72]:
lr = LogisticRegression(random_state=0, max_iter=100)
lr.fit(X_train, train_enc_labels)
pred = lr.predict(X_test)

In [73]:
print(classification_report(test_enc_labels, pred))

              precision    recall  f1-score   support

           0       0.90      0.90      0.90     15103
           1       0.99      1.00      0.99     13717
           2       0.90      0.86      0.88      7783
           3       0.84      0.89      0.86      1390
           4       0.89      0.97      0.93      5672
           5       0.92      0.68      0.78      4265
           6       0.00      0.00      0.00        24
           7       0.90      0.95      0.92     36238
           8       0.83      0.90      0.86      1734
           9       0.94      0.78      0.85      5125
          10       0.82      0.91      0.86      7444
          11       0.77      0.51      0.62      5473
          12       0.81      0.91      0.86      2865
          13       1.00      0.82      0.90        62
          14       0.92      0.93      0.92     17110
          15       0.57      0.03      0.06       134

    accuracy                           0.90    124139
   macro avg       0.81   

In [74]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3), analyzer='word', binary=False)
# Создаем the Bag-of-Words модель
X_train = tfidf_vectorizer.fit_transform(train_tok)

In [75]:
X_test = tfidf_vectorizer.transform(test_tok)

In [76]:
lr = LogisticRegression(random_state=0, max_iter=100)
lr.fit(X_train, train_enc_labels)
pred = lr.predict(X_test)

In [77]:
print(classification_report(test_enc_labels, pred))

              precision    recall  f1-score   support

           0       0.94      0.49      0.65     15103
           1       0.99      0.48      0.64     13717
           2       0.95      0.79      0.86      7783
           3       0.82      0.95      0.88      1390
           4       0.98      0.18      0.30      5672
           5       0.79      0.77      0.78      4265
           6       0.00      0.00      0.00        24
           7       0.51      1.00      0.67     36238
           8       0.86      0.64      0.73      1734
           9       0.94      0.78      0.85      5125
          10       0.91      0.73      0.81      7444
          11       0.93      0.18      0.30      5473
          12       0.76      0.97      0.85      2865
          13       0.00      0.00      0.00        62
          14       0.98      0.52      0.68     17110
          15       0.00      0.00      0.00       134

    accuracy                           0.68    124139
   macro avg       0.71   

# ВЫВОД

из теггеров лучше всех себя показал BigramTagger, но по сравнению с countbase-подходами они показали меньшую accurasy. Из countbase-подходов лучше всех Countvectoriser. Метрики лучше на символах, хуже на словах. В целом датасет нуждается в дополнительной чистке. Нужно убрать предлоги, возможно цифры и т.д.
